In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
!pip install folium
import folium

from bs4 import BeautifulSoup
print("Libraries imported.")

Libraries imported.


## read the table using pandas

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
# df

### exclude any rows containing "Not assigned" in the borough column

In [3]:
exclude = ["Not assigned"]
df = df[~df["Borough"].isin(exclude)]
# df

### deal with duplicate postal codes

In [4]:
temp1 = df.groupby("Postcode").agg(lambda x:','.join(set(x)))

In [5]:
# temp1

### set "Not assigned" neighborhoods to the borough assignment

In [6]:
temp1.loc[temp1["Neighbourhood"]=="Not assigned", "Neighbourhood"]=temp1.loc[temp1["Neighbourhood"]=="Not assigned", "Borough"]

In [7]:
# temp1

## get the shape of the dataframe to obtain the number of rows

In [8]:
temp1.shape

(103, 2)

# Read the provided CSV file as a pandas dataframe

In [9]:
df2 = pd.read_csv("http://cocl.us/Geospatial_data")

In [10]:
# df2

# use the geospatial data with the original dataframe

In [41]:
temp1["Latitude"]=df2["Latitude"].values
temp1["Longitude"]=df2["Longitude"].values
# temp1

In [12]:
temp1.shape

(103, 4)

***

# Segmentation and Clustering

### create a map of Toronto with neighborhoods superimposed on top

In [38]:
address = "Toronto, Canada"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

In [39]:
location

Location(Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada, (43.653963, -79.387207, 0.0))

In [40]:
# add markers to map
for lat, lng, borough, neighbourhood in zip(temp1['Latitude'], temp1["Longitude"], temp1["Borough"], temp1["Neighbourhood"]):
    label = "{},{}".format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## get nearby venue info from foursquare

In [17]:
uri = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}".format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude)

In [18]:
results = requests.get(uri).json()

In [19]:
# results

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### turn the venue info into dataframe

In [21]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Karine's,Breakfast Spot,43.653699,-79.390743
4,Rolltation,Japanese Restaurant,43.654918,-79.387424


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list = []
    for name, lat, lng, in zip(names, latitudes, longitudes):
        # print(name)
        
        # create uri
        uri = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}".format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius)
    
        # make the GET request
        results = requests.get(uri).json()["response"]["groups"][0]["items"]
    
        # return only relevant info
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
        nearby_venues.columns = ['Neighborhood',
                                'Neighborhood Latitude',
                                'Neighborhood Longitude',
                                'Venue',
                                'Venue Latitude',
                                'Venue Longitude',
                                'Venue Category',]

    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=temp1["Neighbourhood"],
                                latitudes=temp1["Latitude"],
                                longitudes=temp1["Longitude"],)
# toronto_venues

## Analyze Each Neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix="", prefix_sep="")
# toronto_onehot
# toronto_venues["Neighborhood"]


In [25]:
# add neighborhood column back to dataframe
toronto_onehot["Neighborhood"] = toronto_venues["Neighborhood"]

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Malvern,Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,

In [26]:
toronto_onehot.shape

(1345, 232)

### Group rows by neighborhood and take mean of the frequency of occurrence

In [42]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
# toronto_grouped

### Get the top 5 most common venues for each neighborhood

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to the number of top venues
columns = ["Neighborhood"]
for ind in np.arange(num_top_venues):
    try:
        
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))
        
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted["Neighborhood"] = toronto_grouped["Neighborhood"]

for ind in np.arange(toronto_grouped.shape[0]):
    
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,Richmond,King",Café,Hotel,Asian Restaurant,Seafood Restaurant,Sushi Restaurant,Lounge,Steakhouse,Speakeasy,Smoke Shop,Restaurant
1,Agincourt,Latin American Restaurant,Breakfast Spot,Lounge,Skating Rink,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
2,"Albion Gardens,Silverstone,Mount Olive,Thistle...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Fast Food Restaurant,Beer Store,Sandwich Place,Empanada Restaurant,Electronics Store,Eastern European Restaurant
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Women's Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
4,"Bedford Park,Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pharmacy,Pizza Place,Pub,Café,Butcher


# Cluster Neighborhoods

In [29]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop("Neighborhood", 1)

# run kmeans clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 2, 2, 1], dtype=int32)

## create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [30]:
neighborhoods_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)

toronto_merged = temp1

# merge toronto_grouped with toronto_data to add lat/long for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighbourhood")

toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postcode,,,,,,,,,,,,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497,0.0,Golf Course,Bar,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711,1.0,Spa,Breakfast Spot,Electronics Store,Medical Center,Intersection,Rental Car Location,Mexican Restaurant,Diner,Ethiopian Restaurant,Empanada Restaurant
M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Bank,Gas Station,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Bakery,Diner,Discount Store


# Visualize the resulting clusters

In [37]:
import math
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colros = []
for lat, lon, poi, cluster in zip(toronto_merged["Latitude"], toronto_merged["Longitude"], toronto_merged["Neighbourhood"], toronto_merged["Cluster Labels"]):
    label = folium.Popup(str(poi) + " Cluster " + str(cluster), parse_html=True)
    if math.isnan(cluster):
        cluster = 0
    else:
        cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7
    ).add_to(map_clusters)
    
map_clusters